In [1]:
! pip install transformers
! pip install torch torchvision torchaudio

In [19]:
# MODEL = 'fine_tuned/NER/ACD_5epochs'
# MODEL = 'fine_tuned/NER/ACD'
MODEL = 'fine_tuned/NER/ACD_10epochs'

MODEL_PATH = '../../models/word_embeddings/'

MODEL_PATH += MODEL

In [20]:
import os, json
from transformers import AutoConfig

config_name = "config.json"

config_file = os.path.join(MODEL_PATH, config_name)
config_json = json.load(open(config_file))

config = AutoConfig.from_pretrained(
    MODEL_PATH,
    id2label=config_json["id2label"],
    label2id=config_json["label2id"]
)

labels = []
labels = list(config.id2label.values())

print(config.id2label)
print(config.id2label.values())
print(config.label2id.values())


{'0': 'B-Anatomy', '1': 'B-Chemical', '2': 'B-Disease', '3': 'I-Anatomy', '4': 'I-Chemical', '5': 'I-Disease', '6': 'O'}
dict_values(['B-Anatomy', 'B-Chemical', 'B-Disease', 'I-Anatomy', 'I-Chemical', 'I-Disease', 'O'])
dict_values([0, 1, 2, 3, 4, 5, 6])


In [21]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

# tokenizer = BertTokenizer.from_pretrained(MODEL_PATH, do_lower_case=False)

tokenizer = AutoTokenizer.from_pretrained(
                MODEL_PATH,
                use_fast=False)

# model = BertForTokenClassification.from_pretrained(
#     MODEL_PATH,
#     num_labels=len(labels),
#     output_attentions = False,
#     output_hidden_states = False,
#     config = config
# )

model = AutoModelForTokenClassification.from_pretrained(
        MODEL_PATH,
#         from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config)
#         cache_dir=model_args.cache_dir)

In [23]:
import torch
import numpy as np

# test_sentence = 'Doctor, I am feeling chest pain since yesterday. The pain is continuous and is located just in the middle of my chest, worsening when I breathe and when I lay down on my bed. I suffer from arterial hypertension and smoke 20 cigarettes every day. My father had a “heart attack” at my age and I am very worried about it.'
# test_sentence = " Her eye is green. A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the eyes, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes. Alprazolan was administred."
# test_sentence = 'Recently, we found that therapy with selegiline and L-dopa was associated with selective systolic orthostatic hypotension which was abolished by withdrawal of selegiline. '
# test_sentence = 'The cells were examined in a Zeiss LSM 510 laser scanning microscope equipped with a Plan - Apochromate 63x/1.4 oil immersion objective.'
# test_sentence = 'Torsade de pointes ventricular tachycardia during  low dose intermittent dobutamine treatment in a patient with dilated cardiomyopathy and congestive heart failure.'
# test_sentence = 'A DNA molecule 
test_sentence = """"
@NURSE
> Doctor, we have a man (51 years old) who entered the emergency department reporting chest pain. His vital signs are ABP: 144x92mmHG; HR: 78bpm; RR: 21rpm; Temp: 37oC; O2Sat: 98%.
@PATIENT
> Doctor, I am feeling chest pain since yesterday. The pain is continuous and is located just in the middle of my chest, worsening when I breathe and when I lay down on my bed. I suffer from arterial hypertension and smoke 20 cigarettes every day. My father had a "heart attack" at my age and I am very worried about it.
<b>PHYSICAL EXAMINATION</b> <br> The cardiac and pulmonary auscultation are normal; chest pain does not worse with palpation of the thorax; there is no jugular stasis nor lower limb edema.
Recently, we found that therapy with selegiline and L-dopa was associated with selective systolic orthostatic hypotension which was abolished by withdrawal of selegiline. 
The cells were examined in a Zeiss LSM 510 laser scanning microscope equipped with a Plan - Apochromate 63x/1.4 oil immersion objective.
 """
# test_sentence = 'Doctor, I am feeling a very strong chest pain in the middle of my chest. I have arterial hypertension but I don’t take my medication every day.'
# test_sentence = 'Doctor, I am feeling a very strong chest pain in the middle of my chest. It has started 2 hours ago, but its intensity isn’t decreasing. It started suddenly and I am also feeling pain in my back and my neck. The pain is continuous, and it did not worse when I am breathing. I have arterial hypertension but I don’t take my medication every day, I am sorry! I don’t smoke, and I don’t know If I have other diseases. My father has also arterial hypertension.'
# test_sentence = """Doctor, I am feeling a very strong chest pain in the middle of my chest.
# It started suddenly and I am also feeling pain in my back and my neck.
# I have arterial hypertension but I don’t take my medication every day.
#     """



tokenized_sentence = tokenizer.encode(test_sentence)
# input_ids = torch.tensor([tokenized_sentence]).cuda()
input_ids = torch.tensor([tokenized_sentence])

with torch.no_grad():
    output = model(input_ids)
# print(output)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
print(labels)
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(labels[label_idx])
        new_tokens.append(token)
for token, label in zip(new_tokens, new_labels):
    print("{}\t\t{}".format(label, token))


['B-Anatomy', 'B-Chemical', 'B-Disease', 'I-Anatomy', 'I-Chemical', 'I-Disease', 'O']
O		[CLS]
O		"
O		@
O		NURSE
O		>
O		Doctor
O		,
O		we
O		have
O		a
O		man
O		(
O		51
O		years
O		old
O		)
O		who
O		entered
O		the
O		emergency
O		department
O		reporting
B-Disease		chest
I-Disease		pain
O		.
O		His
O		vital
O		signs
O		are
O		ABP
O		:
O		144x92mmHG
O		;
O		HR
O		:
O		78bpm
O		;
O		RR
O		:
O		21rpm
O		;
O		Temp
O		:
O		37oC
O		;
O		O2Sat
O		:
O		98
O		%
O		.
O		@
O		PATIENT
O		>
O		Doctor
O		,
O		I
O		am
O		feeling
B-Disease		chest
I-Disease		pain
O		since
O		yesterday
O		.
O		The
B-Disease		pain
O		is
O		continuous
O		and
O		is
O		located
O		just
O		in
O		the
O		middle
O		of
O		my
O		chest
O		,
O		worsening
O		when
O		I
O		breathe
O		and
O		when
O		I
O		lay
O		down
O		on
O		my
O		bed
O		.
O		I
O		suffer
O		from
B-Disease		arterial
I-Disease		hypertension
O		and
O		smoke
O		20
O		cigarettes
O		every
O		day
O		.
O		My
O		father
O		had
O		a
O		"
B-Disease		heart
I-Disease		attack
O		"
O